# YOLOv8 Classification Training and InferenceThis notebook demonstrates how to train a YOLOv8 classification model using the dataset that is already laid out in YOLO style folders (`train` and `valid`). The code also shows how to perform inference on new images once training has completed.

## 1. Environment SetupInstall the [Ultralytics](https://docs.ultralytics.com) package that provides the YOLOv8 implementation. It is safe to re-run this cell; it will upgrade to the latest version if required.

In [ ]:
%pip install --quiet ultralytics

## 2. Prepare the Classification DatasetAlthough the images and labels are stored in a YOLO detection format (`images/` + `labels/`), every image only belongs to a single class. We can therefore reorganize the files into the folder structure expected by YOLOv8's classification trainer. The code below creates symbolic links so the conversion is fast and does not duplicate data.

In [ ]:
from pathlib import Pathimport shutilproject_root = Path('.')yolo_root = project_root  # dataset already lives in the project rootsplits = {    'train': yolo_root / 'train' / 'train',    'val': yolo_root / 'valid' / 'valid',}cls_root = project_root / 'classification_dataset'if cls_root.exists():    shutil.rmtree(cls_root)for split_name, split_path in splits.items():    images_dir = split_path / 'images'    labels_dir = split_path / 'labels'    target_split_dir = cls_root / split_name    target_split_dir.mkdir(parents=True, exist_ok=True)    for label_path in labels_dir.glob('*.txt'):        candidate = images_dir / f"{label_path.stem}.jpg"        if candidate.exists():            image_path = candidate        else:            # Fall back to any image that matches the stem with a different suffix            matches = list(images_dir.glob(f"{label_path.stem}.*"))            if not matches:                raise FileNotFoundError(f"Could not find image for label {label_path}")            image_path = matches[0]        with label_path.open() as f:            first_line = f.readline().strip()        if not first_line:            raise ValueError(f"Empty label file: {label_path}")        class_id = first_line.split()[0]        class_dir = target_split_dir / class_id        class_dir.mkdir(parents=True, exist_ok=True)        target_path = class_dir / image_path.name        if target_path.exists() or target_path.is_symlink():            target_path.unlink()        target_path.symlink_to(image_path.resolve())print(f"Classification dataset prepared at: {cls_root.resolve()}")

## 3. Define the Dataset ConfigurationYOLOv8 classification can infer class names directly from the folder structure. For convenience we still create a mapping so that predictions are easy to interpret.

In [ ]:
class_id_to_name = {    '0': 'crazing',    '1': 'inclusion',    '2': 'patches',    '3': 'pitted_surface',    '4': 'rolled_in_scale',    '5': 'scratches',}names_path = cls_root / 'class_names.yaml'names_path.write_text(''.join(f"{k}: {v}" for k, v in class_id_to_name.items()))print(names_path.read_text())

## 4. Train a YOLOv8 Classification ModelThe following cell fine-tunes the lightweight `yolov8n-cls.pt` checkpoint on the prepared dataset. Adjust hyperparameters such as `epochs`, `imgsz`, or `batch` to fit your resources. Training results (metrics, loss curves, and weights) are stored inside the `runs/classify/` directory.

In [ ]:
from ultralytics import YOLOmodel = YOLO('yolov8n-cls.pt')results = model.train(    data=str((cls_root / 'train').resolve()),    val=str((cls_root / 'val').resolve()),    epochs=50,    imgsz=224,    batch=64,)results

## 5. Evaluate and Run InferenceOnce training finishes, the best weights are saved under `runs/classify/train*/weights/best.pt`. Load those weights to perform inference on one or more images. The `predict` call returns prediction objects with probabilities for each class.

In [ ]:
from pathlib import Pathbest_model_path = Path(results.save_dir) / 'weights' / 'best.pt'classifier = YOLO(best_model_path)sample_images = [    next((cls_root / 'val' / '0').glob('*.jpg')),    next((cls_root / 'val' / '1').glob('*.jpg')),]predictions = classifier.predict(source=sample_images, save=True)for pred in predictions:    top1 = max(pred.probs, key=lambda p: p.conf)    print(f"Image: {pred.path}\nTop-1 prediction: {class_id_to_name[str(int(top1.cls))]} (confidence={top1.conf:.2%})\n")

## 6. Next Steps* Adjust the augmentation and optimization hyperparameters to achieve better accuracy.* Export the trained model to ONNX, TensorRT, CoreML, etc., via `model.export(format='onnx')`.* Integrate the classifier into larger pipelines for automated quality control.